In [461]:
from scipy.io import loadmat
from scipy.stats import zscore
from decimal import *
import pandas as pd
import numpy as np
import datetime as dt
import os
import sys
import math
import pandas as pd
import numpy as np
from pprint import pprint
import json
import requests
import datetime
from datetime import date
import os
import xmltodict
from pandas.io.json import json_normalize

In [462]:
initial_value = "Anne_v5e_"

#v1: no colors, initial run to make sure the code works and generates the rough glyph design
#v2: colors added, topologies fixed
#v3: refinement, added facet
#v4: change oura score rods to spheres with cylinders, change location of food glyphs
#v5a: new clock face design, too close together but snowflake looking
#v5b: clock face design flat, evenly spaced out
#v5c: rotate the root sphere, stand up clock and examine each day
#v5d: line up all 14 days, clock face is vertical
#v5e: rods are hidden

# Food DataFrame

In [463]:
dffd = pd.read_csv("servings.csv")

In [464]:
dffd = dffd.set_index(pd.to_datetime(dffd['Day']))


In [465]:
dffd = dffd.loc['2020-07-14':'2020-07-27']


In [466]:
dffd = dffd.reset_index(drop=True)

In [467]:
dffd['Time'] = pd.to_datetime(dffd['Time'])
dffd['mealtime'] = [d.time() for d in dffd['Time']]

In [468]:
df3 = dffd.groupby(['Day', 'mealtime']).sum()

In [469]:
df3 = df3[['energy', 'protein', 'fat', 'carbs', 'fiber', 'sugar']]

In [470]:
tmlist = []
for index, row in df3.iterrows():
    totalmeal = row['protein'] + row['carbs'] + row['fat']
    tmlist.append(totalmeal)

In [471]:
df3['totalmeal'] = tmlist
#df3['totalmeal'] = df3['totalmeal'].fillna(0)

In [472]:
df3['p'] = df3['protein']/ df3['totalmeal']
df3['ft'] = df3['fat']/ df3['totalmeal']
df3['c'] = df3['carbs']/ df3['totalmeal']
df3['s'] = df3['sugar']/ df3['carbs']
df3['fb'] = df3['fiber']/ df3['carbs']

In [473]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor

    return interp_fn

In [474]:
for column in df3[['p', 'ft', 'c']]:
    columnSeriesObj = df3[column]
    col_min = df3[column].min() 
    col_max = df3[column].max() 
    col_list = df3[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, 1.84, 2.8) 
    col_interp = [scalar(x) for x in col_list] 
    df3[(str(column) + str('_scale_xy'))] = col_interp

In [475]:
for column in df3[['p', 'ft', 'c']]:
    columnSeriesObj = df3[column]
    col_min = df3[column].min() 
    col_max = df3[column].max() 
    col_list = df3[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .5, 1.48) 
    col_interp = [scalar(x) for x in col_list] 
    df3[(str(column) + str('_scale_z'))] = col_interp

In [476]:
# cal_chunk is every  100 calories per meal
df3['cal_chunk'] = df3['energy']/100
df3['cal_chunk'] = df3['cal_chunk'].round(0).astype(int)

In [477]:
df3 = df3.reset_index()

In [478]:
df3['date'] = pd.to_datetime(df3['Day'])

In [479]:
df3['mtime'] = df3['mealtime'].astype(str)

In [480]:
df3['dtime'] = df3['date'].astype(str)

In [481]:
df3['fulldate'] = pd.to_datetime(df3['dtime'] + ' ' + df3['mtime'])

In [482]:
df3['hour'] = df3['fulldate'].dt.hour

In [483]:
dfenergy = df3.groupby(['Day']).sum()

In [484]:
caltotal = []
for i in dfenergy['energy']:
    if i >= 2000:
        x = True
    else:
        x = False
    caltotal.append(x)

In [485]:
len(caltotal)

14

## df4 blood glucose

In [486]:
dfblood = pd.read_csv("blood_glucose.csv")

In [487]:
dfg = dfblood.loc[dfblood['type'] == 'glucose']
dfg = dfg[['value', 'date', 'time', 'gki']]
col_list = ['bg', 'date', 'time', 'gki']
dfg.columns = col_list
#dfg.head()

In [488]:
dfk = dfblood.loc[dfblood['type'] == 'ketone']
dfk = dfk[['value', 'date', 'time']]
col_list = ['k', 'date', 'time']
dfk.columns = col_list
#dfk.head()

In [489]:
col_min = dfg['bg'].min() 
col_max = dfg['bg'].max() 
col_list = dfg['bg'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .8, 2) 
col_interp = [scalar(x) for x in col_list] 
dfg['bg_scale'] = col_interp

In [490]:
col_min = dfg['gki'].min() 
col_max = dfg['gki'].max() 
col_list = dfg['gki'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .03, .58) 
col_interp = [scalar(x) for x in col_list] 
dfg['gki_scale'] = col_interp

In [491]:
col_min = dfk['k'].min() 
col_max = dfk['k'].max() 
col_list = dfk['k'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .03, .58) 
col_interp = [scalar(x) for x in col_list] 
dfk['k_scale'] = col_interp

In [492]:
dfg['time'] = pd.to_datetime(dfg['time'])

In [493]:
dfk = dfk.fillna(0)
dfk['time'] = pd.to_datetime(dfk['time'])

In [494]:
#dfsurvey.loc[dfsurvey["Are_you_in_isolation"] == 'Yes', 'Are_you_in_isolation'] = 1
dfk.loc[(dfk['time'] > '07:00:00') & (dfk['time'] < '09:00:00'), 'time_num'] = 8
dfk.loc[(dfk['time'] > '09:00:00') & (dfk['time'] < '11:00:00'), 'time_num'] = 10
dfk.loc[(dfk['time'] > '11:00:00') & (dfk['time'] < '13:00:00'), 'time_num'] = 12
dfk.loc[(dfk['time'] > '13:00:00') & (dfk['time'] < '15:00:00'), 'time_num'] = 14
dfk.loc[(dfk['time'] > '15:00:00') & (dfk['time'] < '17:00:00'), 'time_num'] = 16
dfk.loc[(dfk['time'] > '17:00:00') & (dfk['time'] < '19:00:00'), 'time_num'] = 18

In [495]:
dfk['time_num'] = dfk['time_num'].astype(int)
dfk['t_num'] = pd.to_datetime(dfk['time_num'], format = '%H')

In [496]:
dfk['t_num_time'] = dfk['t_num'].dt.time

In [497]:
klist = []
for index, row in dfk.iterrows():
    x = '2020-01-01' + ' ' + str(row['t_num_time'])
    klist.append(x)

In [498]:
dfk['tmt'] = pd.to_datetime(klist)

In [499]:
dfk['tmtstamp'] = (dfk['tmt'].astype(int) / 10**9)

In [500]:
col_list = dfk['tmtstamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [501]:
dfk['y4'] = col_interp

In [502]:
dfg.loc[(dfg['time'] > '07:00:00') & (dfg['time'] < '09:00:00'), 'time_num'] = 8
dfg.loc[(dfg['time'] > '09:00:00') & (dfg['time'] < '11:00:00'), 'time_num'] = 10
dfg.loc[(dfg['time'] > '11:00:00') & (dfg['time'] < '13:00:00'), 'time_num'] = 12
dfg.loc[(dfg['time'] > '13:00:00') & (dfg['time'] < '15:00:00'), 'time_num'] = 14
dfg.loc[(dfg['time'] > '15:00:00') & (dfg['time'] < '17:00:00'), 'time_num'] = 16
dfg.loc[(dfg['time'] > '17:00:00') & (dfg['time'] < '19:00:00'), 'time_num'] = 18

In [503]:
dfg = dfg.fillna(0)

In [504]:
dfg['time_num'] = dfg['time_num'].astype(int)
dfg['t_num'] = pd.to_datetime(dfg['time_num'], format = '%H')

In [505]:
dfg['t_num_time'] = dfg['t_num'].dt.time

In [506]:
glist = []
for index, row in dfg.iterrows():
    x = '2020-01-01' + ' ' + str(row['t_num_time'])
    glist.append(x)

In [507]:
dfg['tmt'] = pd.to_datetime(glist)

In [508]:
dfg['tmtstamp'] = (dfg['tmt'].astype(int) / 10**9)

In [509]:
col_list = dfg['tmtstamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [510]:
dfg['y4'] = col_interp

In [511]:
df4 = dfg.merge(dfk,on = ['date', 'time_num'], how = 'outer')

In [512]:
df4['date'] = pd.to_datetime(df4['date'])

In [513]:
df4 = df4.loc[(df4['date'] > '2020-07-14') & (df4['date'] < '2020-07-28')]

In [514]:
df4['y4'] = df4['y4_x']

In [515]:
df4['hour'] = df4['time_num']
df4 = df4.drop(['time_num'], axis =1)

## df5 Workouts 

In [516]:
dfw = pd.read_csv("14dworkouts.csv")

In [517]:
llist = []
for index, row in dfw.iterrows():
    x = '2020-01-01' + ' ' + str(row['time_start'])
    llist.append(x)

In [518]:
dfw['dtstart'] = pd.to_datetime(llist)

In [519]:
dfw['@workoutActivityType'].value_counts()

HKWorkoutActivityTypeTraditionalStrengthTraining    8
HKWorkoutActivityTypeYoga                           6
Name: @workoutActivityType, dtype: int64

In [520]:
dfw.rename(columns = {'@workoutActivityType': 'type'}, inplace=True)

In [521]:
dfw.loc[dfw['type'] == 'HKWorkoutActivityTypeTraditionalStrengthTraining', 'type'] = 'lifting'

In [522]:
dfw.loc[dfw['type'] == 'HKWorkoutActivityTypeYoga', 'type'] = 'yoga'

In [523]:
dfw['wcal_chunk'] = dfw['@totalEnergyBurned']/25
dfw['wcal_chunk'] = dfw['wcal_chunk'].round(0).astype(int)

In [524]:
dfw['@creationDate'] = pd.to_datetime(dfw['@creationDate'])

In [525]:
dfw['date'] = dfw['@creationDate'].dt.date
dfw['hour'] = dfw['dtstart'].dt.hour

In [526]:
df5 = pd.DataFrame()

In [527]:
df5 = dfw[['date','type', 'wcal_chunk', 'hour']]
df5.head()

,date,type,wcal_chunk,hour
0,2020-07-15,yoga,5,6
1,2020-07-15,lifting,4,6
2,2020-07-16,yoga,5,6
3,2020-07-16,lifting,7,6
4,2020-07-17,yoga,5,5


## df1 Oura data

In [528]:
with open('oura_2020-08-19_AG.json') as json_data:
    dataJ = json.load(json_data)
    
sleep = json_normalize(dataJ, 'sleep')
dfs = pd.DataFrame(sleep)

readiness = json_normalize(dataJ, 'readiness')
dfr = pd.DataFrame(readiness)

activity = json_normalize(dataJ, 'activity')
dfa = pd.DataFrame(activity)

In [529]:
df = dfa.merge(dfr, on='summary_date')

In [530]:
df = df.merge(dfs, on = 'summary_date')
df.rename( columns={'Unnamed: 0':'index'}, inplace=True )

In [531]:
df['bedstart'] = pd.to_datetime(df['bedtime_start'])
df['sum_date'] = pd.to_datetime(df['summary_date'])

In [532]:
df['date'] = df['sum_date'].dt.date

In [533]:
df = df.set_index(pd.DatetimeIndex(df['date']))

In [534]:
df1 = df.loc['2020-07-14':'2020-07-27']

In [535]:
df1 = df1.reset_index(drop=True)

In [536]:
df1.replace({0:0.001}, inplace = True)
df1.fillna(0.001, inplace=True)

In [537]:
def Reverse(lst): 
    lst.reverse() 
    return lst 

In [538]:
for column in df1[['score_x', 'score', 'score_y']]:
    col_min = df1[column].min() 
    col_max = df1[column].max() 
    col_list = df1[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .6, 2.4) 
    col_interp = [scalar(x) for x in col_list] 
    df1[(str(column) + str('_scale'))] = col_interp

In [539]:
xlist = list(range(-140, 140, 20))

In [540]:
print((xlist))

[-140, -120, -100, -80, -60, -40, -20, 0, 20, 40, 60, 80, 100, 120]


In [541]:
#xlist = [xlist]*2
#xaxis = [y for x in xlist for y in x]

In [542]:
ylist = [0] * 14

In [543]:
df1['xroot'] = xlist
df1['yroot'] = ylist
df1['zroot'] = 3

In [544]:
df1['bedstart'] = df1['bedstart'].dt.tz_localize(tz='UTC')

In [545]:
df1['bedstart'] = df1['bedstart'].dt.tz_convert(tz='America/Los_Angeles')

In [546]:
df1['bedtime'] = df1['bedstart'].dt.time

In [547]:
df1['bed_start'] = df1['bedstart'].dt.hour

In [548]:
dfcheck = pd.merge(left = df1, right = df5, how = 'left', left_on = 'date', right_on = 'date')

In [549]:
wkoutday = set(df5['date'].unique())
df1['wout'] = df1['date'].map(lambda x: True if x in wkoutday else False)

In [550]:
df1['caltotal'] = caltotal

In [551]:
cwlist = []
for index, row in df1.iterrows():
    if (row['wout'] == True) & (row['caltotal'] == False):
        x = 1
    elif (row['wout'] == False) & (row['caltotal'] == True):
        x = 4
    elif (row['wout'] == False) & (row['caltotal'] == False):
        x = 2
    elif (row['wout'] == True) & (row['caltotal'] == True):
        x = 3
    cwlist.append(x)

In [552]:
df1['calwork'] = cwlist

In [553]:
bedlist = []
for index, row in df1.iterrows():
    if (row['bed_start'] < 22) & (row['bed_start'] > 14):
        x = 1
    elif (row['bed_start'] == 23) | (row['bed_start'] == 22):
        x = 2
    elif (row['bed_start'] >= 0) | (row['bed_start'] < 2):
        x = 3
    elif (row['bed_start'] >= 2) & (row['bed_start'] < 13) :
        x = 4
    bedlist.append(x)

In [554]:
df1['btchunk'] = bedlist

## df6 survey data

In [555]:
dfsd = pd.read_csv('focus_survey_change.csv')

In [556]:
dfsd.head(2)

,Timestamp,work_type,focus_quality,engagement_rating,company_benefit,comment
0,7/14/20 8:12,admin,4,4,5,prepping forms
1,7/14/20 10:00,creative_generative,4,5,5,personal glyph design


In [557]:
dfsd.work_type.value_counts()

creative_generative            40
admin                          14
with_people                    14
fucking_off                     8
consuming_learning_strategy     6
consuming_learing_strategy      2
Name: work_type, dtype: int64

In [558]:
dfsd = dfsd.fillna(0)

In [559]:
dfsd['datestamp'] = pd.to_datetime(dfsd['Timestamp'])

In [560]:
dfsd['date'] = dfsd['datestamp'].dt.date

In [561]:
dfsd['time_s'] = dfsd['datestamp'].dt.time

In [562]:
dfsd['hour'] = dfsd['datestamp'].dt.hour

In [563]:
df6 = pd.DataFrame()

In [564]:
df6 = dfsd[['work_type', 'comment', 'date', 'hour', 'focus_quality', 'engagement_rating', 'company_benefit']]

## Start df7  timing nodes

In [565]:
hrs_list = ['2020-01-01 04:00:00', '2020-01-01 06:00:00', '2020-01-01 08:00:00', '2020-01-01 10:00:00', '2020-01-01 12:00:00', '2020-01-01 14:00:00', '2020-01-01 16:00:00', '2020-01-01 18:00:00', '2020-01-01 20:00:00', '2020-01-01 22:00:00', '2020-01-02 00:00:00', '2020-01-02 02:00:00', '2020-01-02 04:00:00']

In [566]:
dfhrs = pd.to_datetime(hrs_list)

In [567]:
dfhrs2 = (dfhrs.astype(int)/10**9)

In [568]:
# hrs_list = values for every 2 hours from 04 am to next day at 04 am
hrs_list = dfhrs2.values.tolist()

In [569]:
scalar = make_interpolater(1577851200, 1577937600, -120, 105) 
col_interp = [scalar(x) for x in hrs_list] 
Rcol_interp = Reverse(col_interp)

In [570]:
df7 = pd.DataFrame()
df7['time_toroid'] = Rcol_interp

In [571]:
#df7

In [572]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 18
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [573]:
#personal glyph node files
mn = str(initial_value) + "personalglyph_node.csv"

#met_1min tag files
mt = str(initial_value) + "personalglyph_tag.csv"

In [574]:
# Open ANTz Node file for writing
fout = open(mn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(mt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,700,250,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel1toroids = 1
numlevel2toroids = 2

for index, row in df1.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 6
    selected = 0
    branch_level = 1
    child_id = id
    scale_x = 1.8
    scale_y = scale_x
    scale_z = scale_x 
    translate_x = row['xroot']
    translate_y = row['yroot']
    translate_z = row['zroot']
    rotate_x = 90
    rotate_y = 90
    rotate_z = 180
    ratio = .1
    line_width= 1
    geometry = 3
    topo = 10
    hide = 0
    if row['calwork'] == 1:
        color_r = 166
        color_g = 31
        color_b = 81
    elif row['calwork'] == 2:
        color_r = 242
        color_g = 220
        color_b = 224
    elif row['calwork'] == 3:
        color_r = 242
        color_g = 216
        color_b = 87
    elif row['calwork'] == 4:
        color_r = 140
        color_g = 67
        color_b = 3
    color_a = 255
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "Date: " + str(row['date'])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level1objectid = id
    # toroid holding rods & time marker object
    for j in range(numlevel2toroids):    
        if j == 0: 
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            scale_x = 5.8
            scale_y = scale_x
            scale_z = scale_x 
            translate_x = 180
            translate_y = 0
            translate_z = 0
            rotate_x = 90
            rotate_y = 0
            rotate_z = 0
            ratio = .11
            line_width= 1
            geometry = 7
            topo = 3
            hide = 0
            if row['btchunk'] == 1:
                color_r = 191
                color_g = 181
                color_b = 184
            elif row['btchunk'] == 2:
                color_r = 164
                color_g = 166
                color_b = 156
            elif row['btchunk'] == 3:
                color_r = 76
                color_g = 89
                color_b = 79
            elif row['btchunk'] == 4:
                color_r = 53
                color_g = 64
                color_b = 58
            color_a = 255
            record_id = id 

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            # Output to Tag file
            tagtext = ""
            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
            ftag.write(tagstring)
            taginc += 1

            level2objectid = id

            numlevel3toroids = 3
            #k cones with oura summary scores
            for k in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                if k == 0:
                    translate_x = 0
                    scale_x = row['score_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    color_r = 102
                    color_g = 0
                    color_b = 0
                    tagtext = "Sleep Score: " + str(row['score'])
                elif k == 1:
                    translate_x = -120
                    scale_x = row['score_x_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    color_r = 51
                    color_g = 0
                    color_b = 102
                    tagtext = "Activity Score: " + str(row['score_x'])
                elif k == 2: 
                    translate_x = 120
                    scale_x = row['score_y_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    color_r = 0
                    color_g = 51
                    color_b = 102
                    tagtext = "Readiness Score: " + str(row['score_y'])
                translate_y = 165
                translate_z = 570
                ratio = .1
                line_width= 1
                geometry = 5
                topo = 2
                rotate_x = 0
                rotate_y = 0
                color_a = 255
                hide = 0
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

                # Output to Tag file
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1    

            # cone indicating midnight
            numlevel3toroids = 1
            for j in range(numlevel3toroids):     
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                scale_x = 1
                scale_y = scale_x
                scale_z = scale_x 
                translate_x = 0
                translate_y = 0
                translate_z = 255
                rotate_x = 0
                rotate_y = 0
                rotate_z = 0
                ratio = .1
                line_width= 1
                geometry = 5
                topo = 2
                color_r = 191
                color_g = 164
                color_b = 115
                color_a = 255
                hide = 0
                record_id = id 

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                
            numlevel3toroids = 24
            # rods for each hour
            hlist = []
            for h in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                translate_x = 360 - ((360/numlevel3toroids)*h) 
                translate_y = 0
                translate_z = 370
                scale_x = 4.35
                scale_y = scale_x
                scale_z = scale_x
                ratio = .06
                color_r = 89
                color_g = 89
                color_b = 89
                geometry = 19
                topo = 6
                hide = 1
                record_id = id

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                
                # Output to Tag file
                tagtext = str("H: ") + str(h)
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1 
                
                
                x = id
                hlist.append(x)
                
            listh = list(range(24))
            #hlist = parent_id's, and listh = 0 to 23 for h time
            for i, h in zip(hlist, listh):
                #blood glucose glyphs
                for i4, row4 in df4.loc[(df4['date'] == row['date']) & (df4['hour'] == h)].iterrows():
                    id += 1
                    data = id
                    type = 5
                    parent_id = i
                    selected = 0
                    branch_level = 4
                    translate_x = -172
                    translate_y = 0
                    translate_z = 0
                    scale_x = row4['bg_scale']
                    scale_y = scale_x
                    scale_z = scale_x
                    geometry = 3
                    topo = 2
                    color_r = 191
                    color_g = 15
                    color_b = 15
                    hide = 0
                    record_id = id
                    tagtext = "Blood Glucose: " + str(row4['bg'])

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    # Output to Tag file
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1 

                #survey cylinders
                numlevel4toroids = 3
                for i6, row6 in df6.loc[(df6['date'] == row['date']) & (df6['hour'] == h)].iterrows():
                    for jj in range(numlevel4toroids):
                        if jj == 0:
                            id += 1
                            data = id
                            type = 5
                            parent_id = i
                            selected = 0
                            branch_level = 4
                            translate_x = -142
                            translate_y = 0
                            translate_z = 0
                            geometry = 19
                            topo = 4
                            if row6['focus_quality'] == 1:
                                scale_x = .5
                                scale_y = .5
                                scale_z = scale_x
                            elif row6['focus_quality'] == 2:
                                scale_x = .67
                                scale_y = 1
                                scale_z = scale_x
                            elif row6['focus_quality'] == 3:
                                scale_x = .85
                                scale_y = 1.5
                                scale_z = scale_x
                            elif row6['focus_quality'] == 4:
                                scale_x = 1.05
                                scale_y = 2
                                scale_z = scale_x
                            elif row6['focus_quality'] == 5:
                                scale_x = 1.25
                                scale_y = 2.5
                                scale_z = scale_x
                            if row6['work_type'] == 'admin':
                                color_r = 217
                                color_g = 201
                                color_b = 173
                            elif row6['work_type'] == 'creative_generative':
                                color_r = 242
                                color_g = 92
                                color_b = 132
                            elif row6['work_type'] == 'with_people':
                                color_r = 4
                                color_g = 191
                                color_b = 157
                            elif row6['work_type'] == 'fucking_off':
                                color_r = 242
                                color_g = 159
                                color_b = 5
                            elif row6['work_type'] == 'consuming_learning_strategy':
                                color_r = 191
                                color_g = 73
                                color_b = 150
                            elif row6['work_type'] == 'consuming_learing_strategy':
                                color_r = 191
                                color_g = 73
                                color_b = 150
                            hide = 0
                            record_id = id
                            tagtext = "Work Type: " + str(row6['work_type']) + "/ Focus Quality: " + str(row6['focus_quality'])

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            # Output to Tag file
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 

                        elif jj == 1:
                            id += 1
                            data = id
                            type = 5
                            parent_id = i
                            selected = 0
                            branch_level = 4
                            translate_x = -130
                            translate_y = 0
                            translate_z = 0
                            geometry = 19
                            topo = 4
                            if row6['engagement_rating'] == 1:
                                scale_x = .5
                                scale_y = .5
                                scale_z = scale_x
                            elif row6['engagement_rating'] == 2:
                                scale_x = .67
                                scale_y = 1
                                scale_z = scale_x
                            elif row6['engagement_rating'] == 3:
                                scale_x = .85
                                scale_y = 1.5
                                scale_z = scale_x
                            elif row6['engagement_rating'] == 4:
                                scale_x = 1.05
                                scale_y = 2
                                scale_z = scale_x
                            elif row6['engagement_rating'] == 5:
                                scale_x = 1.25
                                scale_y = 2.5
                                scale_z = scale_x
                            if row6['work_type'] == 'admin':
                                color_r = 217
                                color_g = 201
                                color_b = 173
                            elif row6['work_type'] == 'creative_generative':
                                color_r = 242
                                color_g = 92
                                color_b = 132
                            elif row6['work_type'] == 'with_people':
                                color_r = 4
                                color_g = 191
                                color_b = 157
                            elif row6['work_type'] == 'fucking_off':
                                color_r = 242
                                color_g = 159
                                color_b = 5
                            elif row6['work_type'] == 'consuming_learning_strategy':
                                color_r = 191
                                color_g = 73
                                color_b = 150
                            elif row6['work_type'] == 'consuming_learing_strategy':
                                color_r = 191
                                color_g = 73
                                color_b = 150
                            hide = 0
                            record_id = id
                            tagtext = "Engagement Rating: " + str(row6['engagement_rating'])

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            # Output to Tag file
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 


                        elif jj == 2:
                            id += 1
                            data = id
                            type = 5
                            parent_id = i
                            selected = 0
                            branch_level = 4
                            translate_x = -118
                            translate_y = 0
                            translate_z = 0
                            geometry = 19
                            topo = 4
                            hide = 0
                            if row6['company_benefit'] == 1:
                                scale_x = .5
                                scale_y = .5
                                scale_z = scale_x
                            elif row6['company_benefit'] == 2:
                                scale_x = .67
                                scale_y = 1
                                scale_z = scale_x
                            elif row6['company_benefit'] == 3:
                                scale_x = .85
                                scale_y = 1.5
                                scale_z = scale_x
                            elif row6['company_benefit'] == 4:
                                scale_x = 1.05
                                scale_y = 2
                                scale_z = scale_x
                            elif row6['company_benefit'] == 5:
                                scale_x = 1.25
                                scale_y = 2.5
                                scale_z = scale_x
                            if row6['work_type'] == 'admin':
                                color_r = 217
                                color_g = 201
                                color_b = 173
                            elif row6['work_type'] == 'creative_generative':
                                color_r = 242
                                color_g = 92
                                color_b = 132
                            elif row6['work_type'] == 'with_people':
                                color_r = 4
                                color_g = 191
                                color_b = 157
                            elif row6['work_type'] == 'fucking_off':
                                color_r = 242
                                color_g = 159
                                color_b = 5
                            elif row6['work_type'] == 'consuming_learning_strategy':
                                color_r = 191
                                color_g = 73
                                color_b = 150
                            elif row6['work_type'] == 'consuming_learing_strategy':
                                color_r = 191
                                color_g = 73
                                color_b = 150
                            record_id = id
                            tagtext = "Company Benefit: " + str(row6['company_benefit'])

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            # Output to Tag file
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 

                #workout objects
                for i5, row5 in df5.loc[(df5['date'] == row['date']) & (df5['hour'] == h)].iterrows():
                    id += 1
                    data = id
                    type = 5
                    parent_id = i
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = -88
                    translate_y = 0
                    translate_z = 0
                    scale_x = 2.7
                    scale_y = scale_x
                    scale_z = scale_x
                    ratio = .1
                    line_width= 1
                    rotate_x = 90
                    rotate_y = 0
                    if row5['type'] == 'yoga':
                        geometry = 3
                        color_r = 180
                        color_g = 196
                        color_b = 217
                    elif row5['type'] == 'lifting':
                        geometry = 15
                        color_r = 4
                        color_g = 157
                        color_b = 191
                    topo = 2
                    color_a = 255
                    hide = 0
                    record_id = id 
                    texture_id= 0
                    tagtext = "Workout type: " + str(row5['type']) + "/ # of cal's: " + str(row5['wcal_chunk'])

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    # Output to Tag file
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1

                    level4objectid = id 
                    numlevel5toroids = row5['wcal_chunk']
                    for ii in range(numlevel5toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level4objectid
                        selected = 0
                        branch_level = 5
                        child_id = id
                        translate_x = -90
                        translate_y = ((60/numlevel5toroids)*(ii+1))+(30)
                        translate_z = 0
                        color_r = 242
                        color_g = 75
                        color_b = 75
                        ratio = .1
                        scale_x = 1.3 
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 16
                        topo = 5
                        line_width= 1
                        color_a = 255
                        hide = 0
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)


                # food glyphs: carbs
                for i3, row3 in df3.loc[(df3['date'] == row['date']) & (df3['hour'] == h)].iterrows():
                    id += 1
                    data = id
                    type = 5
                    parent_id = i
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = -35
                    translate_y = 0
                    translate_z = 0
                    scale_x = row3['c_scale_xy']
                    scale_y = scale_x
                    scale_z = row3['c_scale_z']
                    ratio = .1
                    line_width= 1
                    rotate_x = 0
                    rotate_y = 0
                    geometry = 19
                    topo = 4
                    color_r = 245
                    color_g = 115
                    color_b = 54
                    color_a = 255
                    hide = 0
                    record_id = id 
                    texture_id= 0
                    tagtext = "Carbs (g): " + str(row3['carbs']) 

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    # Output to Tag file
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1

                # food glyphs: fat
                for i3, row3 in df3.loc[(df3['date'] == row['date']) & (df3['hour'] == h)].iterrows():
                    id += 1
                    data = id
                    type = 5
                    parent_id = i
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = -20
                    translate_y = 0
                    translate_z = 0
                    scale_x = row3['ft_scale_xy']
                    scale_y = scale_x
                    scale_z = row3['ft_scale_z']
                    ratio = .1
                    line_width= 1
                    rotate_x = 0
                    rotate_y = 0
                    geometry = 19
                    topo = 4
                    color_r = 4
                    color_g = 120
                    color_b = 120
                    color_a = 255
                    hide = 0
                    record_id = id 
                    texture_id= 0
                    tagtext = "Fat (g): " + str(row3['fat']) 

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    # Output to Tag file
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1    

                # food glyphs: protein
                for i3, row3 in df3.loc[(df3['date'] == row['date']) & (df3['hour'] == h)].iterrows():
                    id += 1
                    data = id
                    type = 5
                    parent_id = i
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = -5
                    translate_y = 0
                    translate_z = 0
                    scale_x = row3['p_scale_xy']
                    scale_y = scale_x
                    scale_z = row3['p_scale_z']
                    ratio = .1
                    line_width= 1
                    rotate_x = 0
                    rotate_y = 0
                    geometry = 19
                    topo = 4
                    color_r = 194
                    color_g = 33
                    color_b = 33
                    color_a = 255
                    hide = 0
                    record_id = id 
                    texture_id= 0
                    tagtext = "Protein (g): " + str(row3['protein']) 

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    # Output to Tag file
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1 

                # toroid to hold additional data at end of rod
                numlevel4toroids = 1
                for r in range(numlevel4toroids):
                    id += 1
                    data = id
                    type = 5
                    parent_id = i
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = 15
                    translate_y = 0
                    translate_z = 0
                    scale_x = 1
                    scale_y = scale_x
                    scale_z = scale_x
                    ratio = .1
                    line_width= 1
                    rotate_x = 90
                    rotate_y = 0
                    geometry = 7
                    topo = 3
                    color_r = 89
                    color_g = 89
                    color_b = 89
                    color_a = 175
                    hide = 1
                    record_id = id 
                    texture_id= 0 

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    level4objectid = id
                    # bedtime start object

                    if row['bed_start'] == h:
                        numlevel5toroids = 1
                        for b in range(numlevel5toroids):
                            id += 1
                            data = id
                            type = 5
                            parent_id = level4objectid
                            selected = 0
                            branch_level = 5
                            child_id = id
                            translate_x = 180
                            translate_y = 0
                            translate_z = 0
                            scale_x = 12
                            scale_y = scale_x
                            scale_z = scale_x
                            rotate_x = 0
                            geometry = 11
                            topo = 2
                            color_r = 82
                            color_g = 16
                            color_b = 168
                            color_a = 255
                            hide = 0
                            record_id = id

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            tagtext = "Bedtime: " + str(row['bedtime'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1